In [253]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_lgb
importlib.reload(preprocesamiento)
importlib.reload(model_lgb)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

##### Levantamos el dataset con target ya calculado

In [254]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
df.shape

(31362, 19)

##### Preprocesamiento a la minima expresión :)

In [255]:
##### aplicamos OHE
df = preprocesamiento.aplicarOHE(df)
df.shape

(31362, 162)

### Feature Engineering

##### Neural Prophet

In [256]:
neural_prophet_fe = pd.read_csv("./datasets/features_neuralprophet_completo.csv", sep=',', encoding='utf-8')
neural_prophet_fe['ds'] = pd.to_datetime(neural_prophet_fe['ds'], errors='coerce')
# Versión alternativa más robusta:
neural_prophet_fe['periodo'] = neural_prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
neural_prophet_fe = neural_prophet_fe[['periodo', 'product_id', 'trend', "season_yearly", "season_monthly"]]
df = df.merge(neural_prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 165)

##### Prophet

In [257]:
prophet_fe = pd.read_csv("./datasets/prophet_features_tn_zscore.csv", sep=',', encoding='utf-8')
prophet_fe['ds'] = pd.to_datetime(prophet_fe['ds'], errors='coerce')
prophet_fe['periodo'] = prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
prophet_fe = prophet_fe[['periodo', 'product_id', 'trend_add', "yearly_add", "additive_terms", 'trend_mult', 'yearly_mult', 'multiplicative_terms']]
df = df.merge(prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 171)

##### FE Moviles

In [258]:
df = feature_engineering.get_lags(df, "tn", 201912)
df = feature_engineering.get_delta_lags(df, "tn", 24)
df = feature_engineering.get_rolling_means(df, "tn", 201912)
df = feature_engineering.get_rolling_stds(df, "tn", 201912)
df = feature_engineering.get_rolling_mins(df, "tn", 201912)
df = feature_engineering.get_rolling_maxs(df, "tn", 201912)
# df = feature_engineering.get_rolling_medians(df, "tn", 201912)
# df = feature_engineering.get_rolling_skewness(df, "tn", 201912)
# df = feature_engineering.get_autocorrelaciones(df, "tn", 201912)
df.shape

(31362, 626)

##### FE Moviles sobre otras variables

In [259]:
# #  stock final
# df = feature_engineering.get_lagsEspecificos(df, col='stock_final_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='stock_final_zscore')

# #  cust_request_qty
# df = feature_engineering.get_lagsEspecificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='cust_request_qty_zscore')

##### FE Calendario

In [260]:
df = feature_engineering.generar_ids(df)
df = feature_engineering.get_componentesTemporales(df)
df = feature_engineering.get_anomaliasPoliticas(df)
# df = feature_engineering.descomposicion_serie_temporal(df, col='tn')
df.shape

(31362, 651)

##### FE sobre FE

In [261]:
df = feature_engineering.chatGPT_features_serie(df, "tn")
df = feature_engineering.mes_con_feriado(df)
df.shape

(31362, 680)

##### Variables Exogenas

In [262]:
df = feature_engineering.get_dolar(df)
df = feature_engineering.get_IPC(df)
df['ipc'] = df['ipc'].str.replace(',', '.').astype(float)
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)
# df.drop(columns=['ds'], inplace=True)
df.fillna(0, inplace=True) ##### EXPERIMENTAR
df = feature_engineering.correlacion_exogenas(df)
df = feature_engineering.get_mes_receso_escolar(df)
df.shape

(31362, 683)

##### EXP 1

-   


In [263]:
study, models, results = model_lgb.main_entrenar(df, VERSION="v12")

[I 2025-06-25 23:44:36,469] A new study created in memory with name: no-name-8b2f3921-c1eb-41d0-bf9d-853e0e9356f3


Features: ['product_id', 'periodo', 'nacimiento_producto', 'muerte_producto', 'mes_n', 'total_meses', 'producto_nuevo', 'ciclo_de_vida_inicial', 'sku_size', 'stock_final', 'tn', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'cat1_FOODS', 'cat1_HC', 'cat1_PC', 'cat1_REF', 'cat1_nan', 'cat2_ADEREZOS', 'cat2_CABELLO', 'cat2_DENTAL', 'cat2_DEOS', 'cat2_HOGAR', 'cat2_OTROS', 'cat2_PIEL1', 'cat2_PIEL2', 'cat2_PROFESIONAL', 'cat2_ROPA ACONDICIONADOR', 'cat2_ROPA LAVADO', 'cat2_ROPA MANCHAS', 'cat2_SOPAS Y CALDOS', 'cat2_TE', 'cat2_VAJILLA', 'cat2_nan', 'cat3_ACONDICIONADOR', 'cat3_Acond Bebe', 'cat3_Acond Mujer', 'cat3_Aero', 'cat3_Aji Picante', 'cat3_Almidon', 'cat3_Antigrasa', 'cat3_Azul', 'cat3_BARRAS', 'cat3_Baking Bags', 'cat3_Barbacoa', 'cat3_Barra', 'cat3_Baño', 'cat3_Bizcochuelo', 'cat3_CREMA', 'cat3_CUIDADO ESPECIAL', 'cat3_Caldo Cubo', 'cat3_Caldos Granulados', 'cat3_Cara', 'cat3_Cepilllo', 'cat3_Chimichurri', 'cat3_Concentrado', 'cat3_Cremoso', 'cat3_Cristalino', 

[I 2025-06-25 23:46:27,293] Trial 0 finished with value: 15.59830462682434 and parameters: {'max_depth': 9, 'learning_rate': 0.2536999076681772, 'n_estimators': 759, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'reg_alpha': 0.05808361216819946, 'reg_lambda': 0.8661761457749352}. Best is trial 0 with value: 15.59830462682434.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[27]	valid_0's l1: 26.1512
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[31]	valid_0's l1: 28.7795
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[28]	valid_0's l1: 25.1314
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[49]	valid_0's l1: 14.4094
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[50]	valid_0's l1: 14.4708
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[47]	valid_0's l1: 14.4938
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[53]	valid_0's l1: 12.1418
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[118]	valid_0's l1: 12.1514
Tr

[I 2025-06-25 23:47:22,379] Trial 1 finished with value: 15.64481160015418 and parameters: {'max_depth': 13, 'learning_rate': 0.11114989443094977, 'n_estimators': 118, 'min_child_weight': 10, 'subsample': 0.9162213204002109, 'colsample_bytree': 0.6061695553391381, 'reg_alpha': 0.18182496720710062, 'reg_lambda': 0.18340450985343382}. Best is trial 0 with value: 15.59830462682434.


...

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[29]	valid_0's l1: 20.4801
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[86]	valid_0's l1: 20.2676
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[23]	valid_0's l1: 20.7131
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[39]	valid_0's l1: 14.4153
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[47]	valid_0's l1: 14.4145
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[37]	valid_0's l1: 14.5572
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[129]	valid_0's l1: 11.3126
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[60]	valid_0's l1: 12.1284
Training until 

[I 2025-06-25 23:50:38,509] Trial 3 finished with value: 13.988071755152237 and parameters: {'max_depth': 11, 'learning_rate': 0.14447746112718687, 'n_estimators': 279, 'min_child_weight': 6, 'subsample': 0.7962072844310213, 'colsample_bytree': 0.5232252063599989, 'reg_alpha': 0.6075448519014384, 'reg_lambda': 0.17052412368729153}. Best is trial 2 with value: 13.013808985164818.


Did not meet early stopping. Best iteration is:
[279]	valid_0's l1: 4.06419
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[20]	valid_0's l1: 20.5552
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[33]	valid_0's l1: 29.693
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[10]	valid_0's l1: 29.8427
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[26]	valid_0's l1: 15.7732
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[27]	valid_0's l1: 17.5036
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[42]	valid_0's l1: 18.1469
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[15]	valid_0's l1: 21.879
Training until validation scores don't improve for 30 rounds
Earl

[I 2025-06-25 23:51:45,909] Trial 4 finished with value: 17.64469177406094 and parameters: {'max_depth': 4, 'learning_rate': 0.2521267904777921, 'n_estimators': 970, 'min_child_weight': 9, 'subsample': 0.6523068845866853, 'colsample_bytree': 0.5488360570031919, 'reg_alpha': 0.6842330265121569, 'reg_lambda': 0.4401524937396013}. Best is trial 2 with value: 13.013808985164818.


Did not meet early stopping. Best iteration is:
[970]	valid_0's l1: 3.99749

🏆 Mejores parámetros:
max_depth: 8
learning_rate: 0.05958389350068958
n_estimators: 489
min_child_weight: 3
subsample: 0.8059264473611898
colsample_bytree: 0.569746930326021
reg_alpha: 0.29214464853521815
reg_lambda: 0.3663618432936917
boosting_type: gbdt
objective: regression
metric: mae
verbosity: -1
deterministic: True
force_col_wise: True

Entrenando modelo final...
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[489]	valid_0's l1: 3.87246
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[489]	valid_0's l1: 3.84801
Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[489]	valid_0's l1: 3.95444
Ensemble MAE Val: 3.7899

🎯 MAE Test Ensemble: 3.8775

Top 5 mejores trials:
1. MAE: 13.0138
2. MAE: 13.9881
3. MAE: 15.5983
4. MAE: 15.644

In [264]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Supongamos que querés predecir para los períodos 202011 y 202012
periodos_future = [201911, 201912]
df_future = df[df['periodo'].isin(periodos_future)].copy()

# Asegurate de tener las mismas columnas que en entrenamiento
features = [col for col in df.columns if col not in ['target']]

# Predecir
future_preds = model_lgb.predict_with_ensemble(models, df_future, features)

# Si querés evaluar el MAE (solo si tenés target en esos períodos)
if 'target' in df_future:
    mae_future = mean_absolute_error(df_future['target'], future_preds)
    print(f"📊 MAE Future: {mae_future:.4f}")

# Guardar las predicciones
df_future['pred_ensemble'] = future_preds

📊 MAE Future: 29.2295


In [265]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
df_future = df_future[df_future['periodo'] == 201912]
df_future = df_future[df_future['product_id'].isin(productos_ok['product_id'].unique())]
df_future = df_future[['product_id', 'pred_ensemble']]
df_future


,product_id,pred_ensemble
30476,20001,1495.823914
30477,20002,1093.148598
30478,20003,617.772835
30479,20004,473.961938
30480,20005,586.466433
...,...,...
31355,21263,-2.557755
31357,21265,1.405267
31358,21266,-3.076221
31359,21267,-5.054987


In [266]:
df[(df['periodo'] == 201912 ) & (df['product_id'].isin([20375]))][['tn']]

,tn
30808,4.38548


In [267]:
len(df_future[df_future['pred_ensemble'] < 10]['product_id'].unique())

463

In [272]:

len(df_future[df_future['pred_ensemble'] < 81]['product_id'].unique())

705

opcion 1: 
-   menores a 10 tn => promedio 12 meses => 0.291
-   menores a 50 tn => promedio 12 meses => 0.273
-   menores a 81 tn => promedio 12 meses => 0.265
-   menores a 101 tn => promedio 12 meses => 0.275
-   menores a 90 tn => promedio 12 meses => 0.272


In [268]:
# A los productos < 10 tn les pongo el PROMEDIO ULTIMOS 12 MESES
productos_negativos = df_future[df_future['pred_ensemble'] < 90]['product_id'].unique()
df_negativos = feature_engineering.calcular_promedios_12m(productos_negativos, df )
df_future = df_future.merge(df_negativos, on='product_id', how='left')

df_future['promedio_12m'] = df_future['promedio_12m'].fillna(df_future['pred_ensemble'])
df_future



,product_id,pred_ensemble,promedio_12m
0,20001,1495.823914,1495.823914
1,20002,1093.148598,1093.148598
2,20003,617.772835,617.772835
3,20004,473.961938,473.961938
4,20005,586.466433,586.466433
...,...,...,...
775,21263,-2.557755,0.029993
776,21265,1.405267,0.089541
777,21266,-3.076221,0.094659
778,21267,-5.054987,0.092835


opcion 2: pred negativas => promedio 12 meses


In [269]:
# productos_negativos = df_future[df_future['pred_ensemble'] < 0 ]['product_id'].unique()
# df_negativos = feature_engineering.calcular_promedios_12m(productos_negativos, df )
# df_future.drop(columns=['promedio_12m'], inplace=True)
# df_future = df_future.merge(df_negativos, on='product_id', how='left')
# df_future['pred_ensemble'] = df_future['pred_ensemble'].mask(
#     df_future['pred_ensemble'] < 0, 
#     df_future['promedio_12m']
# )

###### Funciono peor que el promedio de los ultimos 12 meses

guardo el archivo

In [ ]:
df_future_c = df_future[['product_id', 'promedio_12m']].copy()
df_future_c.rename(columns={'promedio_12m': 'tn'}, inplace=True)
df_future_c.to_csv("./outputs/predicciones_exp_01_lgb_fe_v6.csv", index=False, sep=',')